In [76]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

All packages successfully loaded


In [77]:
home = 'notebooks/data/' 
file = 'ST000369.xlsx' 

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 181 TOTAL PEAKS: 181
Done!


In [78]:
# Extract PeakList
PeakList = PeakTable['Name']  

# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = Outcomes.values 

# Split Data into Train (2/3) and Test (1/3)
DataTrain, DataTest, YTrain, YTest = train_test_split(DataTable2, Y, test_size=1/3, stratify=Y, random_state=87)

# Extract Train Data 
XTrain = DataTrain[PeakList]                                    
XTrainLog = np.log(XTrain)                                          
XTrainScale, mu, sigma = cb.utils.scale(XTrainLog, method='auto', return_mu_sigma=True)              
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)    

# Extract Test Data
XTest = DataTest[PeakList]                                    
XTestLog = np.log(XTest)                                          
XTestScale = cb.utils.scale(XTestLog, method='auto', mu=mu, sigma=sigma)             
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)  

In [79]:
# set n, the number of most important features, here empirically set to 5
n = 5

In [80]:
# define the model
model = RandomForestClassifier(n_estimators=100, max_features='sqrt', max_depth=3, 
                               criterion='gini', min_samples_split=2, 
                               min_samples_leaf=0.25, max_leaf_nodes=None, n_jobs=None)
# fit the model
model.fit(XTrainKnn, YTrain)
# get importance
importance = model.feature_importances_
importances = list(enumerate(importance))
importances.sort(key=lambda x:x[1], reverse = True)
top_n = importances[:n]
print("The n most important features for the Random Forest Classifier are:")
print()
print("Molecule                    |  feature importance")
print("-------------------------------------------")
for counter, entry in enumerate(top_n):
    mol = PeakTable[PeakTable["Idx"] == (entry[0] + 1)]["Label"].iloc[0]
    print(mol, " " * (26-len(mol)), "|      {:.4f}".format(entry[1]))

The n most important features for the Random Forest Classifier are:

Molecule                    |  feature importance
-------------------------------------------
hypoxanthine                |      0.0404
xylose                      |      0.0303
urea                        |      0.0303
palmitoleic acid            |      0.0303
maltose                     |      0.0303


In [85]:
# define the model
model = SVC(C=5e-9, kernel="linear", degree=3, gamma="auto", probability=True, tol=0.001, max_iter=-1)
# fit the model
model.fit(XTrainKnn, YTrain)
# get importance
importance = model.coef_[0]
importances = list(enumerate(importance))
importances.sort(key=lambda x:x[1], reverse = True)
top_n = importances[:n]
print("The n most important features for the linear SVM are:")
print()
print("Molecule                    |  feature coefficients ")
print("-------------------------------------------")
for counter, entry in enumerate(top_n):
    mol = PeakTable[PeakTable["Idx"] == (entry[0] + 1)]["Label"].iloc[0]
    print(mol, " " * (26-len(mol)), "|      {:.9f}".format(entry[1]))

The n most important features for the linear SVM are:

Molecule                    |  feature coefficients 
-------------------------------------------
glutamic acid               |      0.000000074
arachidonic acid            |      0.000000072
maltose                     |      0.000000069
715929 carbohydrate         |      0.000000067
malic acid                  |      0.000000066


In [83]:
# define the model
model = PCA(n_components=9)
regrmodel = LinearRegression()
k = 9
# fit the model
model.fit(XTrainKnn)
model.x_scores_ = model.transform(XTrainKnn)
regrmodel.fit(model.x_scores_, YTrain)
# get importance
#importance = regrmodel.coef_
importance = model.explained_variance_ratio_
importances = list(enumerate(importance))
importances.sort(key=lambda x:x[1], reverse = True)
top_n = importances[:n]
print("The n most important features for the PCA used in PCR are:")
print()
print("Molecule      |  explained variance ratio ")
print("-------------------------------------------")
for counter, entry in enumerate(top_n):
    mol = PeakTable[PeakTable["Idx"] == (entry[0] + 1)]["Label"].iloc[0]
    print(mol, " " * (12-len(mol)), "|      {:.4f}".format(entry[1]))

The n most important features for the PCA used in PCR are:

Molecule      |  explained variance ratio 
-------------------------------------------
xylose        |      0.1093
xylitol       |      0.0853
xanthine      |      0.0671
valine        |      0.0610
uridine       |      0.0560
